<a href="https://colab.research.google.com/github/liangli217/LLM_learning/blob/main/Agents_with_OpenAI_SDK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><img src="https://drive.google.com/uc?export=view&id=1oCldVicFepwdYbPHcD8Ui4VeyQVJYgut" height="400"><center>

In [8]:
!pip install openai-agents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.9/210.9 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.4/144.4 kB 11.9 MB/s eta 0:00:00


In [9]:
from openai import OpenAI
from typing import List, Optional, Dict, Any
from dataclasses import dataclass
from agents import Agent, Runner, function_tool
import time, random, json

In [10]:
from google.colab import userdata
api_key = userdata.get('OPENAI_API_KEY')
import os
os.environ['OPENAI_API_KEY'] = api_key

#Proper client initialization
client = OpenAI(api_key=os.environ.get('OPENAI_API_KEY'))

In [8]:
model = "gpt-4o"
user_prompt ="What's the best performing ETF?"

In [ ]:
class BaseLLM :

  def __init__(self, model ):
    self.model = model

  def generate(self, user_prompt:str, max_tokens = 1000):

    print("BaseLLM: making single API call...")
    start_time = time.time()
    response = client.completions.create(
        model=self.model,
        max_tokens=max_tokens,
        input =[{"role": "user", "content": user_prompt}]
    )
    end_time = time.time()

    return {
        "content": response.output_text,
        "tokens_used": response.usage.total_tokens,
        "response_time": end_time - start_time
    }


In [ ]:
base_llm = BaseLLM(model)
llm_result = base_llm.generate(user_prompt)

BaseLLM: making single API call...


In [ ]:
print("\n" + "="*50)
print("BASE LLM:",model)
print("="*50)
print(f"Response:")
print(llm_result['content'])
print(f"Tokens: {llm_result['tokens_used']}, Time: {llm_result['response_time']:.2f}s")


BASE LLM: gpt-4o
Response:
Determining the "best" performing ETF can depend on the specific time frame, market conditions, and what you're looking to achieve. Some ETFs may be top performers in the short term but may not be suitable for long-term investments. Common performance metrics include total return over specific periods or performance compared to a benchmark index.

To find the current best-performing ETF, you would typically look at financial news websites or platforms like Morningstar, Bloomberg, or Yahoo Finance. They often have updated lists and reports on ETFs that are currently outperforming others. Remember, past performance doesn't guarantee future results, and it's important to consider factors like fees, investment strategy, and risk when evaluating an ETF.
Tokens: 149, Time: 6.94s


In [ ]:
model = "gpt-5"
user_prompt ="What's the best performing ETF (US only), provide top 3 for the best performance in the YTD?"

In [ ]:
base_llm = BaseLLM(model)
llm_result = base_llm.generate(user_prompt)

BaseLLM: making single API call...


In [ ]:
print("\n" + "="*50)
print("BASE LLM:",model)
print("="*50)
print(f"Response:")
print(llm_result['content'])
print(f"Tokens: {llm_result['tokens_used']}, Time: {llm_result['response_time']:.2f}s")


BASE LLM: gpt-4o
Response:
Buying a home in the U.S. is a multi-step process that typically involves the following stages:

1. **Financial Preparation:**
   - **Credit Score:** Check and improve your credit score if necessary.
   - **Budget:** Determine how much you can afford, including down payment and monthly mortgage payments.
   - **Pre-Approval:** Get pre-approved for a mortgage to understand your price range and prove to sellers you're serious.

2. **Finding a Home:**
   - **Real Estate Agent:** Hire a qualified real estate agent to guide you.
   - **Home Search:** Identify your needs and wants in a home and start searching.
   - **Visit Homes:** Tour potential homes to assess their condition and suitability.

3. **Making an Offer:**
   - **Offer Price:** Decide on the price based on market conditions and the home's value.
   - **Negotiation:** Negotiate terms with the seller, which may include contingencies like inspections.

4. **Due Diligence:**
   - **Inspections:** Conduct

**Reasoning LLM**

In [ ]:
class ReasoningLLM:

  def __init__(self, base_llm: BaseLLM):
    self.basellm = base_llm
    self.conversation_history = []

  def analyze_query(self, query: str):

    analysis_prompt = f"""Analyze this query and determin the best approach to answer it comprehensively:
    Query: "{query}"

    Respond with:
    1. Query type(factual, process comparion, etc.)
    2. Key components to address
    3. Suggested breakdown steps

    Keep resposne concise and structured


    """

    print("Reasoning LLM: Step 1 - Analzying query..")
    response = self.base_llm.generte(analysis_prompt)

    return {"analysis": response["content"], "tokens": response["tokens_used"]}



  def decompose_problem(self, query: str, analysis: str):
    decomposition_prompt =f"""
    Based on this analysis: {analysis}

    Break down the query "{query}" into 3-4 specific sub-question that, when answered together,
    will provide a comprehensive response. List only the questions, one per line.
    """

    print("Reasoning LLM: Step 2- Decomposing problem...")
    response = self.base_llm.generate(decomposition_prompt)

    questions = [q.strip() for q in response['content'].split('\n') if q.strip() and '?' in q]
    return questions[:4] # Limit to 4 questionsj


  def answer_sub_questions(self, question: str) -> str:

    focused_prompt = f"""
    Answer this specific question about selecting top 3 ETF with precise, technical details:
    {question}

    Provide a clear, factual answer focusing only on this aspect.
    """

    print(f"ReasoiningLLM: Step 3 - Answering: {question[:50]}...")
    response = self.base_llm.generate(focused_prompt)
    return response["content"]


  def verify_response(self, question: str, answer: str) -> Dict[str, Any]:
    verification_prompt = f"""

    Question: {question}
    Answer: {answer}

    Evaluate this answer on a scale of 1-10 for:
    1. Accuracy
    2. Completeness
    3. Clarity

    Respond with just three numbers and any critical missing information

    """

    print("ReasoningLLM: Step 4- Verifying response...")
    response = self.base_llm.generate(verification_prompt)
    return {"verification": response["content"]}


  def synthesie_final_response(self, query: str, qa_pairs: List[tuple]) -> str:
      synthesis_prompt =f"""
      Original question: {query}

      Sub-questions and answers with verifications:

        {chr(10).join([
            f"Q: {qa['question']}\nA: {qa['answer']}\nVerification: {qa['verification']}\n"
            for qa in qa_pairs
        ])}

        Synthesize these into a well-structured, comprehensive answer to the original questions.
        Use clear steps/sections and ensure logical flow.
        """

      print("ReasoningLLM: Ste 5 - Synthesizing final response...")
      response = self.base_llm.generate(synthesis_prompt)
      return response["content"]



  def reason(self, query: str) -> str:
    print(f"\n ReasoningLLM: Processing '{query}")
    start_time = time.time()

    # Step 1: analyze query
    analysis = self.analyze_query(query)

    # Step 2: decompose into sub-questions
    sub_questions = self.decompose_problem(query, analysis["analysis"])

    # Step 3&4: Answer and verify each sub-question
    qa_pairs = []

    for question in sub_questions:
      answer = self.answer_sub_questions(question)
      verification = self.verify_response(question, answer)
      qa_pairs.append({"question": question, "answer": answer, "verification": verification["verification"]})

    # Step 5: Synthesize

    final_response = self.synthesie_final_response(query, qa_pairs)

    end_time = time.time()


    return {
        "response": final_response,
        "reasoning_steps": {
            "analysis": analysis["analysis"],
            "sub_questions": sub_questions,
            "qa_pairs": qa_pairs
        },
        "processing_time": end_time - start_time
        }





In [ ]:
user_prompt ="what is the process of buying a home in the US?"

In [ ]:
model = "gpt-4o"
base_llm = BaseLLM(model)
reasoning_llm = ReasoningLLM(base_llm)

print("\n" + "="*50)
print("REASONING LLM:",model)
print("="*50)
reasoning_result = reasoning_llm.reason(user_prompt)
print(f"Final Response:")
print(reasoning_result['response'])
print(f"Sub-questions explored: {len(reasoning_result['reasoning_steps']['sub_questions'])}")
print(f"Time: {reasoning_result['processing_time']:.2f}s")


REASONING LLM: gpt-4o

 ReasoningLLM: Processing 'what is the process of buying a home in the US?
Reasoning LLM: Step 1 - Analzying query..
BaseLLM: making single API call...
Reasoning LLM: Step 2- Decomposing problem...
BaseLLM: making single API call...
ReasoiningLLM: Step 3 - Answering: 1. What steps should I take before starting the ho...
BaseLLM: making single API call...
ReasoningLLM: Step 4- Verifying response...
BaseLLM: making single API call...
ReasoiningLLM: Step 3 - Answering: 2. How do I explore financing options and navigate...
BaseLLM: making single API call...
ReasoningLLM: Step 4- Verifying response...
BaseLLM: making single API call...
ReasoiningLLM: Step 3 - Answering: 3. What should I consider during the home search a...
BaseLLM: making single API call...
ReasoningLLM: Step 4- Verifying response...
BaseLLM: making single API call...
ReasoiningLLM: Step 3 - Answering: 4. What are the important steps in closing the dea...
BaseLLM: making single API call...
ReasoningL

**Simple Agent**

In [14]:
from agents import (
    Agent,
    Runner,
    WebSearchTool,
    function_tool,
    SQLiteSession,
    ModelSettings,
    FunctionTool
)
from agents.agent_output import AgentOutputSchemaBase
from agents.mcp import MCPServerStreamableHttp
from agents.exceptions import ModelBehaviorError
from agents.items import ToolCallItem, ToolCallOutputItem


In [ ]:
model = "gpt-4o"

In [ ]:
user_prompt ="what is the process of buying a home in the US?"

In [ ]:
class SimpleAgent:


  def __init__(self, model, user_location=None, search_context_size=None, instructions=None):


     # configure the web search tool
      web_tool = WebSearchTool(
          user_location = user_location,
          search_context_size = search_context_size
      )

      if instructions is None:
        instructions = (
            "You are a meticulous research agent. "
                "Before answering, outline a brief plan with 3–4 sub-questions. "
                "Use the web_search tool for up-to-date or factual details. "
                "Perform at least TWO web_search calls across DIFFERENT reputable domains. "
                "Cross-check key facts (dates, figures, definitions). "
                "Then write a clear, structured answer with sections and practical recommendations. "
                "Finish with a 'Sources' list (title + URL) covering the items you used."
        )



        self.agent = Agent(
            name= "SimpleAgent",
            instructions = instructions,
            model = model,
            tools = [web_tool]
        )



  def _research_prompt(self, query):
    return f"""

    Task: Answer the user's question comprehensively.

    User question: {query}

    Workflow you must follow:

    1) **Plan**: List 3–4 concise sub-questions you will answer.
    2) **Research**: Use `web_search` at least twice across different domains to gather facts.
    3) **Synthesize**: Combine findings into a cohesive answer that is better than a generic LLM response.
    4) **Verify**: Re-check critical claims and resolve conflicts if sources disagree.
    5) **Present**: Write the final answer with these sections:
      - Summary (2–4 sentences)
      - Key Criteria / How to Choose
      - Top Recommendations (with brief justifications)
      - Smart Variations or Blends (if applicable)
      - Practical Tips (actionable steps)
      - Sources (bullet list: Title — URL)

    Output format: Markdown.
    If uncertainty remains, say what is uncertain and why.
    """


  async def generate(self, query: str):

    print(f"SimpleAgent: Runnign with web_search for '{query}")
    start = time.time()

    result = await Runner.run(self.agent, input = self._research_prompt(query))


    end = time.time()

    return {
        "response": result.final_ouput,
        "processing_time": end - start,
        "tools": self.agent.tools
    }

In [22]:
from IPython.display import HTML, Markdown

In [ ]:
  agent = SimpleAgent(model)
  print("\n" + "="*50)
  print("AGENT:",model)
  print("="*50)
  agent_result = await agent.generate(user_prompt)
  print(f"Final Response:")
  display(Markdown(agent_result['response']))
  print(f"Tools used: {len(agent_result['tools'])}")
  print(f"Time: {agent_result['processing_time']:.2f}s")


AGENT: gpt-4o
SimpleAgent: Runnign with web_search for 'what is the process of buying a home in the US?
Final Response:


Below is my structured response following your specified workflow.

## Plan – Sub-questions to Answer

1. **What are the main steps in the U.S. home-buying process?**  
2. **What key criteria should buyers consider when choosing a home?**  
3. **What top recommendations (e.g., contingencies, loans, professional support) can improve outcomes?**  
4. **What practical, actionable tips help smooth the process?**  

---

## Research 

### First web search — general process and criteria  
- Retrieved step-by-step guidance from HSBC, American Express, Quicken Loans, Zillow, Realtor.com, and First American Bank covering affordability, pre-approval, agent selection, inspections, appraisal, closing, etc. ([us.hsbc.com](https://www.us.hsbc.com/home-loans/10-steps-to-buying-your-own-home/?utm_source=openai))  

### Second web search — specific recommendations and nuances  
- Investopedia’s guide for single buyers highlighted financial thresholds, down payment norms, and recommendation for contingency planning ([investopedia.com](https://www.investopedia.com/guide-to-purchasing-a-home-on-your-own-11702851?utm_source=openai))  
- Wall Street Journal article clarified earnest money norms and contingency importance ([wsj.com](https://www.wsj.com/buyside/personal-finance/what-is-earnest-money-55028b14?utm_source=openai))  
- Chip Gaines’s “Big Four” inspection red flags emphasized which structural issues to watch for ([bhg.com](https://www.bhg.com/chip-gaines-home-buying-red-flags-8691444?utm_source=openai))  

---

## Verify  

Across sources, the home-buying steps are consistent: determine affordability → get pre-approved → choose agent → search → make offer (with contingencies) → inspections/appraisal → closing. Figures like earnest money (1–3%, sometimes flat $1,000) are aligned across WSJ, Quicken Loans, American Express. No significant conflicts detected.

---

## Final Answer

### Summary  
Buying a home in the U.S. follows a well-defined process: assess affordability, secure financing via pre-approval, enlist a real estate agent, search and tour properties, submit an offer with protective contingencies, conduct inspections and appraisal, and complete the purchase at closing. Key considerations include your budget, loan type, property condition, and working with trusted professionals.

---

### Key Criteria / How to Choose

- **Affordability**: Focus on homes costing no more than 3–5× your annual income; ensure housing costs fall within 28–30% of gross income ([discover.com](https://www.discover.com/home-loans/mortgage/10-steps-to-buying-a-home/index.html?utm_source=openai)).  
- **Down Payment & Costs**: Save for a down payment—ideally 20% to avoid PMI—but many first-timers put down 9% on average; also budget 2–7% of purchase price for closing ([investopedia.com](https://www.investopedia.com/guide-to-purchasing-a-home-on-your-own-11702851?utm_source=openai)).  
- **Credit and DTI**: Maintain a credit score of at least ~620 and debt-to-income under 35–43% for better loan prospects ([investopedia.com](https://www.investopedia.com/guide-to-purchasing-a-home-on-your-own-11702851?utm_source=openai)).  
- **Home Condition**: Look closely at roofing, electrical, plumbing, and foundation—Gaines calls these the “Big Four” red flags ([bhg.com](https://www.bhg.com/chip-gaines-home-buying-red-flags-8691444?utm_source=openai)).

---

### Top Recommendations

- **Get Pre-approved—not just pre-qualified**: Pre-approval gives sellers confidence and clarifies your purchasing power (valid for 60–90 days) ([quickenloans.com](https://www.quickenloans.com/learn/steps-to-buying-a-house?utm_source=openai)).  
- **Include key contingencies in offers**: Mortgage, inspection, and appraisal contingencies protect you if problems arise ([quickenloans.com](https://www.quickenloans.com/learn/steps-to-buying-a-house?utm_source=openai)).  
- **Use earnest money strategically**: Typically 1–3% (or ~$1,000) as “good faith”; higher amounts may strengthen offers in competitive markets ([wsj.com](https://www.wsj.com/buyside/personal-finance/what-is-earnest-money-55028b14?utm_source=openai)).  
- **Build a reliable professional team**: Agent, inspector, lender, attorney/title officer—each adds critical oversight and expertise ([investopedia.com](https://www.investopedia.com/guide-to-purchasing-a-home-on-your-own-11702851?utm_source=openai)).

---

### Smart Variations or Blends

- **Loan Types**: Consider FHA loans (3.5% down), conventional, VA, or USDA depending on eligibility and trade-offs ([investopedia.com](https://www.investopedia.com/how-to-apply-for-an-fha-loan-8667663?utm_source=openai)).  
- **Flexible Wish Lists**: Prioritize must-haves over nice-to-haves and remain open to compromise on less critical features ([zillow.com](https://www.zillow.com/learn/10-steps-to-buying-a-home//?utm_source=openai)).

---

### Practical Tips

1. **Start with Budgeting**: Use affordability calculators and check your credit at the three bureaus; fix errors early ([chase.com](https://www.chase.com/personal/mortgage/education/buying-a-home/buy-home-initial-steps.html?utm_source=openai)).  
2. **Pre-Approval Prep**: Gather documentation (ID, pay stubs, tax returns, bank statements, proof of assets) ([americanexpress.com](https://www.americanexpress.com/en-us/credit-cards/credit-intel/steps-to-buying-a-house/?utm_source=openai)).  
3. **Partner with an Agent**: Preferably one familiar with your target neighborhood and well-reviewed ([zillow.com](https://www.zillow.com/learn/10-steps-to-buying-a-home//?utm_source=openai)).  
4. **During Tours, Watch for the “Big Four” Issues**: Foundation, roof, plumbing, electrical—bring a checklist ([bhg.com](https://www.bhg.com/chip-gaines-home-buying-red-flags-8691444?utm_source=openai)).  
5. **Use Contingencies Wisely**: Only waive after confident in inspection/appraisal to avoid losing earnest money ([wsj.com](https://www.wsj.com/buyside/personal-finance/what-is-earnest-money-55028b14?utm_source=openai)).  
6. **Prepare for Closing**: Review Closing Disclosure early, arrange funds for down payment and closing costs (2–7%) ([realtor.com](https://www.realtor.com/advice/buy/10-step-guide-for-first-time-home-buyers/?utm_source=openai)).

---

### Sources

- **10 Steps to Buying Your Own Home** — HSBC Bank USA  
- **9 Steps to Consider When Buying a House** — American Express  
- **How To Buy A House In 11 Simple Steps** — Quicken Loans  
- **10 Most Important Steps to Buying a House** — Zillow  
- **Steps to Buying a House for the First Time** — Realtor.com  
- **Single and Ready to Buy: A Guide to Purchasing a Home on Your Own** — Investopedia  
- **What Is Earnest Money in Home Buying, and How Much Do You Need?** — Wall Street Journal  
- **Chip Gaines Says These Are the 'Big 4' Red Flags to Avoid When Buying a House** — Better Homes & Gardens  
- **How to Apply for an FHA Loan** — Investopedia

Tools used: 1
Time: 13.31s


In [18]:
instructions = """
1) **Plan**: List 3–4 concise sub-questions you will answer.
    2) **Research**: Use `web_search` at least twice across different domains to gather facts.
    3) **Synthesize**: Combine findings into a cohesive answer that is better than a generic LLM response.
    4) **Verify**: Re-check critical claims and resolve conflicts if sources disagree.
    5) **Present**: Write the final answer with these sections:
      - Summary (2–4 sentences)
      - Key Criteria / How to Choose
      - Top Recommendations (with brief justifications)
      - Smart Variations or Blends (if applicable)
      - Practical Tips (actionable steps)
      - Sources (bullet list: Title — URL)
      """

Using async and await:
with async and await, we can run both tasks concurrently using the asyncio library

In [21]:
import asyncio

In [1]:
import time

def task(name):
    print(f"Starting {name}")
    time.sleep(2)
    print(f"Finished {name}")

def main():
    task("A")
    task("B")

main()

Starting A
Finished A
Starting B
Finished B


In [21]:
# Use async when defining an aynchronous function
async def fetch_data():
    print("Fetching data...")
    await asyncio.sleep(2)
    print("Data fetched.")

In [22]:
# if you call it directly,, it just returns a coroutine object.
fetch_data()

<coroutine object fetch_data at 0x78fe0a728640>

In [23]:
# You can only use await inside an async def

async def main():
    await fetch_data()


**Tools**

In [1]:
PAVLOS_FUN_FACTS = [
    "Mozzarella is the most consumed cheese in the U.S., largely due to pizza.",
    "Gruyère and Emmental are classics for fondue thanks to their smooth melt.",
    "Aging boosts sharpness; 12-month cheddar has nuttier, deeper flavors.",
    "High-moisture cheeses (young gouda, fontina, jack) melt especially evenly.",
    "Rind-on bries are edible; the rind adds mushroomy, earthy notes.",
    "Salt helps control moisture and rind formation during cheesemaking.",
    "Browned spots on grilled cheese = Maillard reaction (not caramelization).",
    "American slices are engineered to melt at lower temperatures.",
    "Fresh cheeses (ricotta, chèvre) soften but don’t stretch like mozzarella.",
    "Taleggio’s washed rind brings savory depth and melts well.",
]

In [3]:
def pavlos_fun_fact():
  return {
      'fact': random.choice(PAVLOS_FUN_FACTS),
      "gathered-at": int(time.time())
  }

In [14]:
@function_tool()
async def pavlos_fun_fact_tool():
  return pavlos_fun_fact()


# Test tool
print(pavlos_fun_fact())
print(pavlos_fun_fact())
print(pavlos_fun_fact())

{'fact': 'American slices are engineered to melt at lower temperatures.', 'gathered-at': 1760470178}
{'fact': 'Fresh cheeses (ricotta, chèvre) soften but don’t stretch like mozzarella.', 'gathered-at': 1760470178}
{'fact': 'Fresh cheeses (ricotta, chèvre) soften but don’t stretch like mozzarella.', 'gathered-at': 1760470178}


In [19]:
# System Prompt
system_prompt = """You are a culinary expert specializing in cheese and comfort foods.
When recommending cheeses, consider:
- Melting properties and optimal temperatures
- Flavor profiles (mild to sharp)
- Availability in typical grocery stores
- Practical cooking techniques

Be specific, practical, and explain your reasoning.

Call the pavlos_fun_fact_tool tool 2–3 times and add facts to your response."""

In [15]:
# Build an agent that uses "pavlos_fun_fact_tool"

culinary_expert_agent = Agent(
    name="Culinary Expert",
    instructions = system_prompt,
    model = model,
    tools = [pavlos_fun_fact_tool],

)

In [6]:
# Define the LLM model and question
model = "gpt-4o"
user_prompt = "What's the best cheese for a grilled cheese sandwich?"

In [18]:
print("\n" + "="*50)
print("Culinary Expert Agent (with fun facts tool call):",model)
print("="*50)
agent_result = await Runner.run(culinary_expert_agent, input=user_prompt)
print(f"Final Response:")
display(Markdown(agent_result.final_output))


Culinary Expert Agent (with fun facts tool call): gpt-4o
Final Response:


For a grilled cheese sandwich, the best choices are cheeses with excellent melting properties and a delightful flavor profile. Here are some recommendations:

1. **Cheddar:**
   - **Melting Properties:** Cheddar melts beautifully, becoming gooey and rich. It’s best when used at medium temperatures, around 150-160°F (65-70°C).
   - **Flavor Profile:** It ranges from mild to sharp, allowing for versatility. Sharp cheddar adds a deeper flavor.
   - **Availability:** Easily available in most grocery stores.
   - **Technique:** Grate the cheese for even melting.

2. **Mozzarella:**
   - **Melting Properties:** Mozzarella has a stretchy melt, perfect for that classic cheese pull.
   - **Flavor Profile:** Mild and milky, suitable for blending with sharper cheeses.
   - **Availability:** Readily found in stores.
   - **Technique:** Fresh mozzarella can be sliced or shredded.

3. **Gouda:**
   - **Melting Properties:** High-moisture gouda melts exceptionally evenly, providing a creamy texture.
   - **Flavor Profile:** Mild and nutty, with aged versions offering more robust flavors.
   - **Availability:** Accessible in most cheese sections.
   - **Technique:** Slice thinly to ensure it melts properly.

4. **Fontina:**
   - **Melting Properties:** Known for its superb melt with a creamy consistency.
   - **Flavor Profile:** A gentle butteriness with a hint of earthiness.
   - **Availability:** Generally available in well-stocked grocery stores.
   - **Technique:** Use thin slices for the best results.

5. **Taleggio:**
   - **Melting Properties:** Its high moisture content allows it to melt smoothly.
   - **Flavor Profile:** It has a washed rind that offers a savory depth.
   - **Availability:** Might be found in specialty stores.
   - **Technique:** Pair with milder cheeses for a balanced sandwich.

### Fun Facts:
- Fresh cheeses like ricotta and chèvre soften rather than stretch.
- High-moisture cheeses, such as young gouda and fontina, melt especially evenly.
- Taleggio’s washed rind brings savory depth and it melts well.

For a flavorful and creamy grilled cheese, consider blending two or more cheeses, like sharp cheddar with mozzarella, to achieve a balance of taste and texture.

**MCP**

In [15]:
# MCP Server
pavlos_mcp_url = "https://pavlos-fun-facts.dlops.io/mcp"

from contextlib import AsyncExitStack
from agents.mcp import MCPServerStreamableHttp

In [17]:
async def run_agent(query: str):
  async with AsyncExitStack() as stack:
    mcp_servers= []

    mcp_server = await  stack.enter_async_context(
        MCPServerStreamableHttp(
            name="Pavlos FastMCP",
            params = {"url": pavlos_mcp_url},
            cache_tools_list= True,
            max_retry_attempts = 3,
            client_session_timeout_seconds = 180,

        )
    )
    mcp_servers.append(mcp_server)


    agent = Agent(
        name = "Culinary Expert (MCP)",
        instructions = system_prompt,
        model = model,
        mcp_servers = mcp_servers,

    )
    result = await Runner.run(agent, input = query)

    return result.final_output


In [23]:
out = await run_agent(user_prompt)
display(Markdown(out))

The best cheese for a grilled cheese sandwich balances melting properties and flavor. Here's my recommendation:

### Top Cheese Choices:

1. **Cheddar**
   - **Melting Properties:** Cheddar melts smoothly when shredded, becoming creamy at around 150°F (65°C).
   - **Flavor Profile:** Ranges from mild to sharp. A 12-month aged cheddar can offer nuttier, deeper flavors, perfect for a more sophisticated sandwich.
   - **Availability:** Easily found in most grocery stores.

2. **American Cheese**
   - **Melting Properties:** Known for its excellent meltability due to emulsifiers, turning gooey at low temperatures.
   - **Flavor Profile:** Mild and creamy, often used for the classic taste of grilled cheese.
   - **Availability:** Widely available.

3. **Gruyère**
   - **Melting Properties:** Silky melt, perfect for a luxurious grilled cheese.
   - **Flavor Profile:** Slightly sweet and nutty.
   - **Availability:** Common in many well-stocked grocery stores.
   - **Fun Fact:** Gruyère and Emmental are classics for fondue because of their smooth melt.

### Tips for Making the Perfect Grilled Cheese:

- **Butter the Bread:** Apply butter on the outside for a golden, crispy crust.
- **Layering:** Combine cheeses for complex flavor; for instance, add a slice of Gruyère to sharp cheddar for both smooth melting and sharp taste.
- **Cooking:** Use a medium-low heat to allow the cheese to melt evenly without burning the bread.

### Fun Facts:

- **Gruyère and Emmental** are often used in fondue for their ability to melt smoothly, much like they do in a grilled cheese.
- **Aging and Flavor:** Aging cheddar enhances its sharpness, adding complexity to your sandwich experience.

Selecting the right cheese or combination can elevate your grilled cheese from ordinary to extraordinary!